In [2]:
import pandas as pd
from pathlib import Path
import dask.dataframe as dd
import warnings
from tqdm import tqdm


warnings.filterwarnings('ignore')

/home/nicol/miniconda3/envs/avoidmers/lib/python3.11/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
path = Path("/storage/group/izg5139/default/DNA-Avoidmers/patterns/pattern_merged_abacaba")

In [5]:
model_organisms = [f for f in Path("pattern_maximal_abacaba").glob("*.maximal.txt") if "chm13" in f.name]
model_organisms

[PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr8.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr4.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr12.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr1.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chrX.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr3.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chrM.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr18.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr16.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_words_length_50_seq_chr10.maximal.txt'),
 PosixPath('pattern_maximal_abacaba/chm13v2.0_abacaba_wo

In [8]:
df_all = []

for species in tqdm(model_organisms):
    df = pd.read_csv(species)
    df_all.append(df)
    
df_all = pd.concat(df_all, axis=0)
df_all

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.74it/s]


,seqID,start,end,sequence,length,type
0,chr8,2692,2744,ctaactggtctctgaccttgattattaacggctgcaaccgggaaag...,52,abacaba
1,chr8,2721,2773,ggctgcaaccgggaaagattttattcaccgtcgatgcggccccgag...,52,abacaba
2,chr8,4032,4089,gcctcctgcaccacttaaagtcagagcgccagttattaatccccat...,57,abacaba
3,chr8,4624,4675,ctgacatttttattgcagtgaatgatgattggggcatcttaagaga...,51,abacaba
4,chr8,6142,6194,ttgatgttacttccaatcaaaagctaccatgcaattatttacataa...,52,abacaba
...,...,...,...,...,...,...
208289,chr11,135122238,135122288,gtctgtggccctgccatagaagcctgagttgatccacatgaaacca...,50,abacaba
208290,chr11,135122240,135122294,ctgtggccctgccatagaagcctgagttgatccacatgaaaccaac...,54,abacaba
208291,chr11,135122592,135122643,acagtgcctgggaagcacttggtaaatggcagctgtaattttgctc...,51,abacaba
208292,chr11,135122593,135122647,cagtgcctgggaagcacttggtaaatggcagctgtaattttgctct...,54,abacaba


In [127]:
import re
aba_pattern = re.compile(r'(.+)(.+)\1')
sequences = set()
def is_aba(x):
    N = len(x)
    if N <= 2:
        return False
        
    return any(x[:i] == x[-i:] for i in range(1, (len(x)+1)//2))
    
def zimin_density(x):
    total_aba = 0 
    N = len(x)
    total_s = N * (N+1) // 2
    
    for l in range(1, N+1):
        for i in range(N-l+1):
            chunk = x[i:i+l]
            
            # is_aba = re.search(aba_pattern, chunk)
            if is_aba(chunk):
                total_aba += 1
                
    return total_aba, total_s, total_aba / total_s

In [126]:
zimin_density("gatga")

(3, 15, 0.2)

In [75]:
df_all['sequence'].to_csv("zimin_sequences.txt", header=None, index=False, mode="w")

In [128]:
from tqdm import tqdm; total = {}

for _, row in tqdm(df_all.iterrows(), total=df_all.shape[0]):
    seq = row['sequence']
    den = zimin_density(seq)
    total.update({seq: den})

  0%|▎                                                                                                                                       | 12092/4651253 [00:20<2:13:38, 578.59it/s]


KeyboardInterrupt: 

In [17]:
zimin_density("agaggggaggggatt")

120
120


0.31666666666666665

In [ ]:
!ls avoidmers_coverage_all